The optical Bloch equations (scaled so that $\tau \equiv t \Omega_R$) are:

$\frac{du}{d\tau} = \frac{1}{\Omega_R}(\delta v - \frac{\Gamma}{2} u)$

$\frac{dv}{d\tau} = \frac{1}{\Omega_R}(-\delta u + \Omega_R w - \frac{\Gamma}{2} v)$

$\frac{dw}{d\tau} = \frac{1}{\Omega_R}(-\Omega_R v - \Gamma(w-1))$



In [ ]:
!pip install qutip

In [3]:
from matplotlib import rc
rc('animation', html='jshtml')

In [4]:
import numpy as np
import qutip
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt, animation as anim
from mpl_toolkits.mplot3d import Axes3D
import types

In [8]:
class Bloch:
  """
  A class that solves the optical Bloch equations and plots the result using the 
  QuTiP library. The plot includes both the Bloch vector and the vector
  W = (omega, 0, delta)
  """
  def __init__(self, gamma, delta, omega, t_range, R0):
    self.gamma = gamma #Decay rate
    self.omega = omega #Rabi frequency
    self.delta = delta #Detuning (N.B. this needs to be a function of time)
    self.t_range = t_range #Range of time values
    self.R0 = R0 #Initial Bloch vector

    self.solve_Bloch()



  def solve_Bloch(self):

    def f (t, R):
      #Defining the optical Bloch equations, in the form dR/dt = f(t,R), where R = [u,v,w]
      dRdt = [(self.delta(t) * R[1] - (self.gamma/2)*R[0])/self.omega,
          (-self.delta(t) * R[0] + self.omega * R[2] - (self.gamma/2)*R[1])/self.omega,
          (-self.omega * R[1] - self.gamma*(R[2] - 1))/self.omega]

      return dRdt

    #Solving equations
    self.R = solve_ivp(fun = f, t_span = [self.t_range[0], self.t_range[-1]], t_eval = self.t_range, y0 = self.R0)
    

  def plot (self):
    #Animating the solution
    #Using code from: https://qutip.org/docs/latest/guide/guide-bloch.html

    fig = plt.figure()
    ax = Axes3D(fig, azim=-40, elev=30)
    sphere = qutip.Bloch(axes=ax)

    def animate(i):
      sphere.clear()
      sphere.add_vectors([[self.omega, 0, self.delta(self.R.t[i])],self.R.y[:, i]])  
      sphere.make_sphere()
      return ax

    def init():
      return ax



    self.anim = anim.FuncAnimation(fig, animate, np.arange(np.shape(self.R.y)[1]),
                                  init_func=init, blit=False, repeat=False)


    plt.close()


In the next three cells, I solve the equations in the case of no damping ($\Gamma = 0$) and varying but constant detuning ($\delta = 0, \delta = 1, \delta = 10$). 

As expected, changing the detuning changes the direction about which the Bloch vector precesses (it always precesses about the vector $\mathbf{W} = (\Omega_R, 0, \delta)$, which is also plotted in these animations) and the precession frequency.

In [11]:
system1 = Bloch(gamma = 0, delta = lambda t: 0, omega = 1, t_range = np.linspace(0, 6*np.pi, num = 20), R0= [0, 0, 1])
system1.plot()
system1.anim


<Figure size 360x360 with 0 Axes>

In [ ]:
system2 = Bloch(gamma = 0, delta = lambda t: 1, omega = 1, t_range = np.linspace(0, 6*np.pi, num = 100), R0= [0, 0, 1])
system2.plot()

system2.anim


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
system3 = Bloch(gamma = 0, delta = lambda t: 10, omega = 1, t_range = np.linspace(0, 6*np.pi, num = 100), R0= [0, 0, 1])
system3.plot()

system3.anim

Output hidden; open in https://colab.research.google.com to view.

In the next two cells, I solve the equations in the case of no damping ($\Gamma = 0$) and a time-varying detunung. The first cell has a slowly increasing detuning ($\delta = \frac{20}{2 \pi * 100} t - 10$) while the second cell has a rapidly increasing ($\delta = \frac{20}{2 \pi * 5} t - 10$) detuning.

In [ ]:
system4 = Bloch(gamma = 0, delta = lambda t: (20/(2*np.pi * 100))*t - 10, omega = 1, t_range = np.linspace(0, 200*np.pi, num = 100), R0= [0, 0, 1])
system4.plot()

system4.anim

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
system5 = Bloch(gamma = 0, delta = lambda t: (20/(2*np.pi * 5))*t - 10, omega = 1, t_range = np.linspace(0, 10*np.pi, num = 100), R0= [0, 0, 1])
system5.plot()

system5.anim

Output hidden; open in https://colab.research.google.com to view.

I now solve the equations in the case of some damping ($\Gamma =0.1$) and a constant detuning ($\delta = 1$), showing that damping gives a steady-state solution.

In [ ]:
system6 = Bloch(gamma = 0.1, delta = lambda t: 1, omega = 1, t_range = np.linspace(0, 50*np.pi, num = 200), R0= [0, 0, 1])
system6.plot()

system6.anim

Output hidden; open in https://colab.research.google.com to view.